In [0]:
#read http file
#* Fields: id, date, user, pc, url, content

import pandas as pd
import re
import csv
#load classified domains
domains_dim_df=pd.read_csv(r'E:\Documents\Educacion\Maestría\UP_DataScience\MachineLearning\r4.2\all_users_domains_classified.csv')
leak_domains= set(domains_dim_df.loc[:,'domain'][domains_dim_df['type']=='leaks'])
thief_domains=set(domains_dim_df.loc[:,'domain'][(domains_dim_df['competitor'] == 1) | (domains_dim_df['type'] == 'jobsearch')])
sabotage_domains=set(domains_dim_df.loc[:,'domain'][domains_dim_df['type'].isin (['spyware','downloads','hacking'])])

In [0]:
http_file = open(r'E:\Documents\Educacion\Maestría\UP_DataScience\MachineLearning\r4.2\http.csv', "r")
http_file_out= open(r'E:\Documents\Educacion\Maestría\UP_DataScience\MachineLearning\r4.2\http_out.csv',"w") 
                    
http_df_col=['date','user','domain','leak_risk','thief_risk','sabotage_risk']
with http_file_out:
    writer=csv.writer(http_file_out)
    writer.writerow(http_df_col)
    http_file.readline()
    for line in http_file:
        leak_risk=False
        thief_risk=False
        sabotage_risk=False
        fields=line.split(",")
        http_domain= re.findall('^(?:https?:\/\/)?(?:[^@\/\n]+@)?(?:www\.)?([^:\/?\n]+)',fields[4])[0]
        leak_risk |= http_domain in leak_domains
        thief_risk |= http_domain in thief_domains
        sabotage_risk |= http_domain in sabotage_domains
        http_row=[fields[1],fields[2],http_domain,  1 if leak_risk else 0,1 if thief_risk else 0,1 if sabotage_risk else 0] 
        writer.writerow(http_row)

http_file.close()
http_file_out.close()

In [0]:
import pandas as pd
http_df= pd.read_csv(r'E:\Documents\Educacion\Maestría\UP_DataScience\MachineLearning\r4.2\http_out.csv', parse_dates=['date'])

In [0]:
http_df=pd.merge(http_df, domains_dim_df[['domain','type']],on='domain', how='left')

In [0]:
http_df['day_of_week']=http_df['date'].apply( lambda x: x.weekday()) 
http_df['hour'] = http_df['date'].apply( lambda x: x.hour*3600+ x.minute*60 +x.second) 

In [0]:
def get_risk_out_of_work(row):  
    try:     
        week_end_leak=week_end_thief=week_end_sabotage=0
        off_hour_leak=off_hour_leak=off_hour_leak=0
        if (row.day_of_week >4):            
            week_end_leak= 1 if row.leak_risk else 0
            week_end_thief= 1 if row.thief_risk else 0
            week_end_sabotage= 1 if row.sabotage_risk else 0
        if ( (row.hour < 28800) | (row.hour >61200)):            
            off_hour_leak= 1 if row.leak_risk else 0
            off_hour_thief= 1 if row.thief_risk else 0
            off_hour_sabotage= 1 if row.sabotage_risk else 0
        return week_end_leak,week_end_thief,week_end_sabotage,off_hour_leak,off_hour_leak,off_hour_leak
    except:
        return 0,0,0,0,0,0   


http_df[['week_end_leak','week_end_thief','week_end_sabotage','off_hour_leak','off_hour_thief','off_hour_sabotage']]= http_df.apply(lambda x: get_risk_out_of_work(x),axis=1 ,result_type='expand')


In [0]:
import pandas as pd
http_monthly_activity=http_df.groupby(['user',pd.Grouper(key='date',freq="M")]).agg(
    leak_risk_cnt= ('leak_risk', "sum") ,
    leak_we_cnt= ('week_end_leak', "sum") ,
    leak_off_hr_cnt=('off_hour_leak', "sum") ,
    thief_risk_cnt=('thief_risk', "sum"),  
    thief_we_cnt= ('week_end_thief', "sum") ,
    thief_off_hr_cnt=('off_hour_thief', "sum") ,    
    sabotage_risk_cnt=('sabotage_risk', "sum"),
    sabotage_we_cnt=('week_end_sabotage', "sum"),
    sabotage_off_hr_cnt=('off_hour_sabotage', "sum")    
)

http_monthly_activity=http_monthly_activity.reset_index()
http_monthly_activity['period']=http_monthly_activity['date'].dt.strftime('%Y%m')

In [0]:
http_monthly_activity

,user,period,leak_risk_cnt,leak_we_cnt,leak_off_hr_cnt,thief_risk_cnt,thief_we_cnt,thief_off_hr_cnt,sabotage_risk_cnt,sabotage_we_cnt,sabotage_off_hr_cnt
0,AAE0190,201001,0,0,0,24,0,0,161,0,0
1,AAE0190,201002,0,0,0,5,0,0,200,0,0
2,AAE0190,201003,0,0,0,28,0,0,235,0,0
3,AAE0190,201004,0,0,0,20,0,0,222,0,0
4,AAE0190,201005,0,0,0,13,0,0,186,0,0
...,...,...,...,...,...,...,...,...,...,...,...
15894,ZSL0305,201101,0,0,0,18,0,0,8,0,0
15895,ZSL0305,201102,0,0,0,24,0,0,7,0,0
15896,ZSL0305,201103,0,0,0,10,0,0,7,0,0
15897,ZSL0305,201104,0,0,0,15,0,0,10,0,0


In [0]:
http_df['period']=http_df['date'].dt.strftime('%Y%m')

KeyError: 'date'

In [0]:
http_df.drop(['date'],axis=1, inplace=True)

In [0]:
http_monthly_activity.drop(['date'],axis=1, inplace=True)

In [0]:
http_monthly_activity=http_monthly_activity[['period','user','leak_risk_cnt','leak_we_cnt','leak_off_hr_cnt','thief_risk_cnt','thief_we_cnt','thief_off_hr_cnt','sabotage_risk_cnt','sabotage_we_cnt','sabotage_off_hr_cnt']]

In [0]:
http_pivot=http_df[['user','period','type','domain']].pivot_table( index=['user','period',], columns='type' ,aggfunc='count', fill_value=0)
http_pivot=http_pivot.reset_index()
http_pivot.columns= list (http_pivot.columns.get_level_values(0)[:2])+list( http_pivot.columns.get_level_values(1)[2:])
http_pivot
#http_monthly_activity=pd.merge(http_monthly_activity, http_pivot,on='domain', how='left')

,user,period,adv,astronomy,banking,cars,chat,cooking,dating,downloads,...,trading,travel,updatesites,urlshortener,warez,webmail,webphone,webradio,webtv,wellness
0,AAE0190,201001,135,0,23,0,88,0,1,161,...,0,25,0,0,28,0,3,1,389,0
1,AAE0190,201002,142,0,28,0,89,0,0,200,...,0,38,0,0,32,1,4,1,447,0
2,AAE0190,201003,163,0,42,0,115,0,0,235,...,0,34,0,0,36,0,12,1,560,0
3,AAE0190,201004,157,0,29,0,65,0,1,222,...,0,48,0,0,29,1,8,3,511,0
4,AAE0190,201005,144,0,19,0,81,0,0,186,...,0,34,5,0,24,1,1,7,371,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15894,ZSL0305,201101,1,1,15,0,0,0,1,8,...,0,1,0,0,8,0,3,0,2,0
15895,ZSL0305,201102,1,0,7,0,0,0,1,7,...,0,0,0,0,9,0,3,0,1,0
15896,ZSL0305,201103,4,0,8,0,0,1,3,7,...,0,0,0,2,7,0,3,0,0,0
15897,ZSL0305,201104,4,0,15,0,0,0,1,10,...,0,1,0,0,9,0,2,0,1,0


In [0]:
final_http_risk=pd.merge(http_monthly_activity,http_pivot ,on=['user','period'])

In [0]:
final_http_risk.to_csv(r'E:\Documents\Educacion\Maestría\UP_DataScience\MachineLearning\r4.2\final_http_risk.csv', index=False)

In [0]:
final_http_risk

,period,user,leak_risk_cnt,leak_we_cnt,leak_off_hr_cnt,thief_risk_cnt,thief_we_cnt,thief_off_hr_cnt,sabotage_risk_cnt,sabotage_we_cnt,...,trading,travel,updatesites,urlshortener,warez,webmail,webphone,webradio,webtv,wellness
0,201001,AAE0190,0,0,0,24,0,0,161,0,...,0,25,0,0,28,0,3,1,389,0
1,201002,AAE0190,0,0,0,5,0,0,200,0,...,0,38,0,0,32,1,4,1,447,0
2,201003,AAE0190,0,0,0,28,0,0,235,0,...,0,34,0,0,36,0,12,1,560,0
3,201004,AAE0190,0,0,0,20,0,0,222,0,...,0,48,0,0,29,1,8,3,511,0
4,201005,AAE0190,0,0,0,13,0,0,186,0,...,0,34,5,0,24,1,1,7,371,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15894,201101,ZSL0305,0,0,0,18,0,0,8,0,...,0,1,0,0,8,0,3,0,2,0
15895,201102,ZSL0305,0,0,0,24,0,0,7,0,...,0,0,0,0,9,0,3,0,1,0
15896,201103,ZSL0305,0,0,0,10,0,0,7,0,...,0,0,0,2,7,0,3,0,0,0
15897,201104,ZSL0305,0,0,0,15,0,0,10,0,...,0,1,0,0,9,0,2,0,1,0
